In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
import matplotlib.pyplot as plt
import gvar as gv
import fkfpi_lib as flib

In [2]:
switches = dict()
switches['ensemble'] = ['l1648f211b580m013m065m838','l2448f211b580m0064m0640m828','l3248f211b580m00235m0647m831',
                'l2464f211b600m0170m0509m635','l2464f211b600m0130m0509m635','l2464f211b600m0102m0509m635','l2464f211b600m00507m0507m628','l3264f211b600m00507m0507m628','l4064f211b600m00507m0507m628', 'l4864f211b600m00184m0507m628',
                'l3296f211b630m0074m037m440','l4896f211b630m00363m0363m430']

switches['ansatz'] = dict()
switches['ansatz']['type'] = 'MA' # Type of fit: 'xpt' or 'MA'
switches['ansatz']['truncation'] = 4 # Order to truncate series
switches['ansatz']['FV'] = False # True turns on NLO FV correction
switches['ansatz']['xsb'] = False # True turns on O(a) discretization
switches['ansatz']['alpha'] = False # True turns on O(alpha_s a^2) discretization

In [3]:
# Set priors for Xpt
priors = dict()
priors['L5'] = gv.gvar(0.0, 10.0)
priors['s2'] = gv.gvar(0.0, 10.0)
# Physical parameters from PDG
phys_params = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_params['mpi'] = gv.gvar(139.57018, 0.00035) # mpi +/- [MeV]

In [4]:
data = pd.read_csv('./github_fkfpi_v1.csv')
hisq_params = pd.read_csv('./hisq_params.csv')

In [5]:
fmt_data = flib.format_data(switches,data,hisq_params,priors)
print(fmt_data)

{'x': array([ 1.40435373,  1.37133917,  1.36314289,  1.19618295,  1.16617205,
        1.14647898,  1.11078674,  1.1084827 ,  1.10880589,  1.09039055,
        0.85209774,  0.81368816]), 'y': array([1.1015(14), 1.1281(18), 1.1746(41), 1.0743(11), 1.08987(94),
       1.1017(15), 1.1386(27), 1.1314(21), 1.1279(28), 1.1609(35),
       1.1108(16), 1.1439(17)], dtype=object), 'p': {'L5': 0(10), 's2': 0(10), 'mpi': array([0.23609(29), 0.16543(22), 0.10093(48), 0.24339(15), 0.21438(15),
       0.18862(23), 0.13556(32), 0.13427(17), 0.13402(15), 0.08128(24),
       0.14074(12), 0.097896(63)], dtype=object), 'mka': array([0.40460(25), 0.38691(21), 0.37796(23), 0.34341(14), 0.33309(16),
       0.32416(21), 0.31044(22), 0.31003(17), 0.31018(19), 0.30204(22),
       0.24110(14), 0.228703(91)], dtype=object), 'mss': array([0.51875(17), 0.51495(14), 0.51314(13), 0.42073(13), 0.41971(13),
       0.41803(16), 0.41576(15), 0.41539(12), 0.41557(12), 0.41420(11),
       0.31170(11), 0.309358(70)], dtype=ob

In [6]:
fit = flib.fit_data(switches,fmt_data,phys_params)

[ 0.287744    0.2968656   0.29861515  0.24061925  0.24957308  0.25489956
  0.2667866   0.26771054  0.26767531  0.27438635  0.18456181  0.19581322]
logs: [-0.98228059 -0.95415651 -0.92468879 -1.26200482 -1.27122962 -1.27673726
 -1.26132968 -1.25487778 -1.25567835 -1.22516349 -4.20756046 -4.6464487 ]
[0.287744(0) 0.296866(0) 0.298615(0) 0.240619(0) 0.249573(0) 0.2549(0)
 0.266787(0) 0.267711(0) 0.267675(0) 0.274386(0) 0.184562(0) 0.195813(0)]
logs: [-0.982281(0) -0.954157(0) -0.924689(0) -1.262(0) -1.27123(0) -1.27674(0)
 -1.26133(0) -1.25488(0) -1.25568(0) -1.22516(0) -4.20756(0) -4.64645(0)]
[-0.92587287 -1.64504321 -2.17477466 -0.42094066 -0.49645602 -0.58490482
 -0.85667622 -0.98713882 -1.01559351 -1.17516992  0.09088512  0.07567338]
logs: [        nan         nan         nan         nan         nan         nan
         nan         nan         nan         nan -0.07283107  0.48912618]
[-0.925873(0) -1.64504(0) -2.17477(0) -0.420941(0) -0.496456(0)
 -0.584905(0) -0.856676(0) -0.987139(

/Users/cchang5/Physics/c51/project_fkfpi/fkfpi_lib.py:157: RuntimeWarning: invalid value encountered in log
  c = 3./8.*self.eX2*np.log(self.eX2)
/Users/cchang5/anaconda/envs/pyqcd3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1757: RuntimeWarning: invalid value encountered in slogdet
  sign, logdet = _umath_linalg.slogdet(a, signature=signature)
/Users/cchang5/anaconda/envs/pyqcd3/lib/python3.6/site-packages/lsqfit/__init__.py:627: UserWarning: det(fit.cov) < 0 --- roundoff errors? Try an svd cut.
  warnings.warn('det(fit.cov) < 0 --- roundoff errors? Try an svd cut.')


In [7]:
print(fit)

ValueError: cannot convert float NaN to integer